# 1. Loading CSV to Hive using Spark

In [1]:
# import os
# from cmlbootstrap import CMLBootstrap
from pyspark.sql import SparkSession
# Change to the appropriate Datalake directory location

# DATALAKE_DIRECTORY = os.environ["STORAGE"]
DATALAKE_DIRECTORY = "s3a://go01-demo"

spark = (
  SparkSession.builder.appName("MyApp")
  .config("spark.jars", "/opt/spark/optional-lib/iceberg-spark-runtime.jar")
  .config("spark.sql.hive.hwc.execution.mode", "spark")
  .config( "spark.sql.extensions", "com.qubole.spark.hiveacid.HiveAcidAutoConvertExtension, org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
  .config("spark.sql.catalog.spark_catalog.type", "hive")
  .config( "spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
  .config("spark.yarn.access.hadoopFileSystems", DATALAKE_DIRECTORY)
  .getOrCreate()
  )


Setting spark.hadoop.yarn.resourcemanager.principal to wseol


In [2]:
datafile=spark.read.csv("/home/cdsw/data/raw/preprocessed_flight_data.csv",header=True, inferSchema=True)

In [3]:
datafile.show(5)

+----------+-------------+---------+------+----+----------+----------+---------+--------------+--------+----+----+
|   fl_date|uniquecarrier|flightnum|origin|dest|crsdeptime|crsarrtime|cancelled|crselapsedtime|distance|hour|week|
+----------+-------------+---------+------+----+----------+----------+---------+--------------+--------+----+----+
|2010-09-14|           US|     1628|   ROC| PIT|       715|       820|        1|          65.0|   224.0|   7|37.0|
|2010-09-15|           US|     1628|   ROC| PIT|       715|       820|        1|          65.0|   224.0|   7|37.0|
|2010-09-16|           US|     1628|   ROC| PIT|       715|       820|        1|          65.0|   224.0|   7|37.0|
|2010-02-06|           US|     2608|   BDL| PBI|       700|      1011|        1|         191.0|  1133.0|   7| 5.0|
|2010-08-31|           CO|     1173|   EWR| ORD|      1345|      1519|        1|         154.0|   719.0|  13|35.0|
+----------+-------------+---------+------+----+----------+----------+---------+

In [4]:
spark.sql("drop table if exists airlines.flights_wseol")

Hive Session ID = 82048650-329b-496a-8573-6795b4c1b001


DataFrame[]

In [5]:
datafile.write.saveAsTable("airlines.flights_wseol")

In [6]:
spark.sql("select * from airlines.flights_wseol limit 5").show()

+----------+-------------+---------+------+----+----------+----------+---------+--------------+--------+----+----+
|   fl_date|uniquecarrier|flightnum|origin|dest|crsdeptime|crsarrtime|cancelled|crselapsedtime|distance|hour|week|
+----------+-------------+---------+------+----+----------+----------+---------+--------------+--------+----+----+
|1997-10-17|           DL|     1286|   PVD| ATL|      1725|      2003|        0|         158.0|   903.0|  17|42.0|
|1997-10-29|           DL|     1775|   PVD| ATL|       630|       909|        0|         159.0|   903.0|   6|44.0|
|1997-10-25|           DL|      759|   RDU| ATL|       730|       848|        0|          78.0|   356.0|   7|43.0|
|1997-10-01|           DL|      985|   RDU| ATL|       600|       716|        0|          76.0|   356.0|   6|40.0|
|1997-10-28|           DL|     1112|   PNS| ATL|      1020|      1222|        0|          62.0|   272.0|  10|44.0|
+----------+-------------+---------+------+----+----------+----------+---------+